# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [4]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()

except Exception as e:
    print(e)

#### Create Keyspace

In [5]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)

except Exception as e:
    print(e)

#### Set Keyspace

In [6]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Explanation

I decided to use this design of the table because we need filter the query by sessionId and itemInSession.
This is why i make a sessionId an itemInSession a composite primary key.

In [8]:
first_query = "SELECT artist, song, length FROM artist_song WHERE sessionId = 338 AND itemInSession = 4"

query = """CREATE TABLE IF NOT EXISTS artist_song (
               sessionId int,
               itemInSession int,
               artist text, 
               song text, 
               length decimal, 
               PRIMARY KEY (sessionId, itemInSession))
        """
try:
    session.execute(query)
except Exception as e:
    print(e)

### Explanation

For the same reason that I have explained in the previous step. I have to add the clustering column itemInSession for sort values by this column

In [9]:
second_query = "SELECT artist, song, firstName, lastName FROM music_library WHERE userId = 10 AND sessionId = 182"

query = """CREATE TABLE IF NOT EXISTS music_library (
               userId int,
               itemInSession int,
               sessionId int,
               artist text, 
               song text, 
               firstName text,
               lastName text,
               PRIMARY KEY ((userId, sessionId), itemInSession))
        """
try:
    session.execute(query)
except Exception as e:
    print(e)

### Explanation

In this case I put sessionId as clustering column to get order in our log history

In [10]:
third_query = "SELECT firstName, lastName FROM user_history_table WHERE song = 'All Hands Against His Own'"

query = """CREATE TABLE IF NOT EXISTS user_history_table (
               song text,
               sessionId int,
               firstName text,
               lastName text,
               PRIMARY KEY ((song), sessionId))
        """
try:
    session.execute(query)
except Exception as e:
    print(e)

In [11]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_song (sessionId,itemInSession,artist,song,length)"
        query = query +  " VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]),int(line[3]),line[0],line[9],float(line[5])))

In [12]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library (userId,itemInSession,sessionId,artist,song,firstName,lastName)"
        query = query +  " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]),int(line[3]),int(line[8]),line[0],line[9],line[1],line[4]))

In [13]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_history_table(song,sessionId,firstName,lastName)"
        query = query +  " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[8]), line[1], line[4]))

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [14]:
try:
    rows = session.execute(first_query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

                    

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=Decimal('495.3073'))


In [15]:
try:
    rows = session.execute(second_query)
except Exception as e:
    print(e)

for row in rows:
    print(row)
                    

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [16]:
try:
    rows = session.execute(third_query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(firstname='Sara', lastname='Johnson')
Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')


### Drop the tables before closing out the sessions

In [17]:
## TO-DO: Drop the table before closing out the sessions

In [18]:
query = "drop table user_history_tables"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


Error from server: code=2200 [Invalid query] message="unconfigured table user_history_tables"


In [19]:
query = "drop table music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


In [20]:
query = "drop table artist_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()